In [1]:
import requests
import psycopg2

In [2]:
def create_tables():
    """create tables in PostgreSQL database"""
    commands = (
        """
        CREATE TABLE dim_film (
            id INTEGER PRIMARY KEY,
            title TEXT,
            episode_id INTEGER,
            opening_crawl TEXT,
            director TEXT,
            producer TEXT,
            character_count INTEGER,
            planet_count INTEGER,
            starship_count INTEGER,
            vehicle_count INTEGER,
            species_count INTEGER,
            release_date DATE,
            created TIMESTAMP,
            edited TIMESTAMP
        )
        """,
        """
        CREATE TABLE dim_people (
            id INTEGER PRIMARY KEY,
            name TEXT,
            height INTEGER,
            mass INTEGER,
            hair_color TEXT,
            skin_color TEXT,
            eye_color TEXT,
            birth_year TEXT,
            gender TEXT,
            homeworld TEXT,
            species_count INTEGER,
            vehicle_count INTEGER,
            starship_count INTEGER,
            created TIMESTAMP,
            edited TIMESTAMP
        )
        """,
        """
        CREATE TABLE film_people_map (
            film_id INTEGER REFERENCES dim_film(id) DEFERRABLE INITIALLY DEFERRED, 
            people_id INTEGER REFERENCES dim_people(id) DEFERRABLE INITIALLY DEFERRED
        )
        """
    )

    conn = None
    try:
        # connect to the PostgreSQL server
        conn = psycopg2.connect(host="195.72.11.33",
                    port="5432",
                    user="postgres",
                    password="2o39iudqP1",
                    dbname="starwars")

        cur = conn.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        # commit the changes
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

    print("Successfully created tables")

In [3]:
def insert_sql(table_name, columns, result_list):
    """insert data into tables"""
    sql = f"""INSERT INTO {table_name} ({",".join(columns)}) VALUES ({','.join(["%s"] * len(result_list[0]))})"""

    conn = None
    try:
        conn = psycopg2.connect(host="195.72.11.33",
                        port="5432",
                        user="postgres",
                        password="2o39iudqP1",
                        dbname="starwars"
            )
        cur = conn.cursor()
        #insert rows into table
        cur.executemany(sql, result_list)
        conn.commit()
        cur.close()
    except Exception as e:
        print(("error: ", e))
        print("insert_cmd: ", sql)
    finally:
        if conn is not None:
            conn.close()

In [4]:
def load_films_data():
    swapi_endpoint_film = "https://swapi.dev/api/films/"
    response = requests.get(swapi_endpoint_film)
    film_data = response.json()["results"]

    results = []
    for movie in film_data:
        id = movie['url'].split('/')[-2]
        title = movie['title']
        episode_id = movie["episode_id"]
        opening_crawl = movie["opening_crawl"]
        director = movie["director"]
        producer = movie["producer"]
        character_count = len(movie["characters"])
        planet_count = len(movie["planets"])
        starship_count = len(movie["starships"])
        vehicle_count = len(movie["vehicles"])
        species_count = len(movie["species"])
        release_date = movie["release_date"]
        created_date = movie["created"].split("T")[0]
        created_time = movie["created"].split("T")[1].split(".")[0]
        created = created_date + ' ' + created_time
        edited_date = movie["edited"].split("T")[0]
        edited_time = movie["edited"].split("T")[1].split(".")[0]
        edited = edited_date + ' ' + edited_time

        final_format = [id, title, episode_id, opening_crawl, director,
                        producer, character_count, planet_count, starship_count,
                        vehicle_count, species_count, release_date, created, edited]

        results.append(final_format)

    table_name = "dim_film"
    columns = ["id", "title", "episode_id", "opening_crawl", "director",
                "producer", "character_count", "planet_count", "starship_count", "vehicle_count",
                "species_count", "release_date", "created", "edited"]
    insert_sql(table_name, columns, results)

    print("Successfully loaded films data")


In [5]:
def load_people_data():
    swapi_endpoint_people = "https://swapi.dev/api/people/"
    response_people = requests.get(swapi_endpoint_people)
    people_data = response_people.json()["results"]

    swapi_endpoint_planet = "https://swapi.dev/api/planets/"
    response_planet = requests.get(swapi_endpoint_planet)
    planet_data = response_planet.json()["results"]

    results = []
    planet_name_dict = {}

    for planet in planet_data:
        planet_name = planet["name"]
        url = planet["url"]
        planet_name_dict[url] = planet_name

    for person in people_data:
        id = person['url'].split('/')[-2]
        name = person['name']
        height = person['height']
        mass = person["mass"]
        hair_color = person["hair_color"]
        skin_color = person["skin_color"]
        eye_color = person["eye_color"]
        birth_year = person["birth_year"]
        gender = person["gender"]
        species_count = len(person["species"])
        vehicle_count = len(person["vehicles"])
        starship_count = len(person["starships"])
        created_date = person["created"].split("T")[0]
        created_time = person["created"].split("T")[1].split(".")[0]
        created = created_date + ' ' + created_time
        edited_date = person["edited"].split("T")[0]
        edited_time = person["edited"].split("T")[1].split(".")[0]
        edited = edited_date + ' ' + edited_time
        homeworld_url = person["homeworld"]
        if homeworld_url in planet_name_dict.keys():
            homeworld = planet_name_dict[homeworld_url]
        else:
            response = requests.get(homeworld_url)
            homeworld = response.json()["name"]

        final_format = [id, name, height, mass, hair_color,
                        skin_color, eye_color, birth_year, gender,
                        homeworld, species_count, vehicle_count, starship_count, created, edited]

        results.append(final_format)

    table_name = "dim_people"
    columns = ["id", "name", "height", "mass", "hair_color",
                "skin_color", "eye_color", "birth_year", "gender",
                "homeworld", "species_count", "vehicle_count", "starship_count", "created", "edited"]
    insert_sql(table_name, columns, results)

    print("Successfully loaded people data")

In [6]:
def load_map_data():
    swapi_endpoint_film = "https://swapi.dev/api/films/"
    response_film = requests.get(swapi_endpoint_film)
    film_data = response_film.json()["results"]

    swapi_endpoint_people = "https://swapi.dev/api/people/"
    response_people = requests.get(swapi_endpoint_people)
    people_data = response_people.json()["results"]

    results = []
    for person in people_data:
        person = person["url"]
        for film in film_data:
            film_char = film["characters"]
            if person in film_char:
                map = (film["url"].split('/')[-2], person.split('/')[-2])
                results.append(map)
    table_name = "film_people_map"
    columns = ["film_id", "people_id"]
    insert_sql(table_name, columns, results)
    print("Successfully loaded map data")

In [7]:
if __name__ == "__main__":
    create_tables()
    load_films_data()
    load_people_data()
    load_map_data()

connection to server at "195.72.11.33", port 5432 failed: Operation timed out
	Is the server running on that host and accepting TCP/IP connections?

Successfully created tables
